In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Sep 23 15:19:51 2018

@author: sihambelgadi
"""

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import io
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
 

df = pd.read_csv('500-1000.csv')
test=pd.read_csv('Testing_data_500.csv')


# encoder = LabelBinarizer()
encoder = LabelEncoder()
X = df.iloc[:,5:-1].values
Xt = test.iloc[:,5:-1].values
y = encoder.fit_transform(df.iloc[:,0])
yt= encoder.fit_transform(test.iloc[:,0])
print(X.shape)
print(y.shape)
print(Xt.shape)
print(yt.shape)

FileNotFoundError: File b'model-500-1000.csv' does not exist

In [2]:
import keras
import time
from keras.layers import Dense, Input, Dropout
from keras.models import Model,Sequential
from keras.wrappers import scikit_learn
from sklearn.model_selection import cross_validate, KFold

from keras import regularizers
import keras.backend as K

from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

encoder = LabelEncoder()
X = df.iloc[:,5:-1].values
y = encoder.fit_transform(df.iloc[:,0])
X, X_test, y, y_test = train_test_split(X, y, train_size=0.75, random_state=42)

X_orig = X
Y_orig = y

print(X.shape)
print(y.shape)
def data():
    return X, y


def model(train, dropout, neurons1, neurons2, reg):
    
    X_orig, y_orig = train
        
    def neuralnet():
        #inputs = Input(shape=(5000,))
        kreg = regularizers.l2(reg)
        ACT = 'relu'
        '''
        o = a = Dense(neurons1, activation=ACT, kernel_regularizer=kreg)(inputs)
        o = Dropout(dropout)(o)
        o = Dense(neurons2, activation=ACT, kernel_regularizer=kreg)(o)
        o = Dropout(dropout)(o)
        o = Dense(4, activation='softmax', kernel_regularizer=kreg)(o)
        z = o

        m = Sequential(inputs=inputs, outputs=z)
        '''
        model = Sequential()
        model.add(Dense(neurons1, input_dim=5000, activation=ACT, kernel_regularizer=kreg))
        model.add(Dropout(dropout))
        model.add(Dense(neurons2, activation=ACT, kernel_regularizer=kreg))
        model.add(Dropout(dropout))
        model.add(Dense(4, activation='softmax', kernel_regularizer=kreg))


        model.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])
        return model

    # m.fit(X, y, epochs=1000,  steps_per_epoch=100)

    K.clear_session()
    clf = scikit_learn.KerasClassifier(build_fn=neuralnet, epochs=10,steps_per_epoch=250, verbose=1)

    k_fold = KFold(n_splits=3, shuffle=False)
    
    acc = []
    f1  = []
    pre = []
    rec = []
    for seed in range(5):
        X, y = shuffle(X_orig, y_orig, random_state=seed)
        cv = cross_validate(clf, X, y, cv=k_fold, scoring=['accuracy', 'f1_weighted','precision_weighted','recall_weighted'])
        print(cv)
        a = cv['test_accuracy']
        f = cv['test_f1_weighted']
        p = cv['test_precision_weighted']
        r = cv['test_recall_weighted']
        
        print("a: {}, f1: {}, pre: {}, rec: {}".format(str(a), str(f), str(p), str(r)))
        acc.append(a)
        f1.append(f)
        pre.append(p)
        rec.append(r)
        
    
    K.clear_session()
    
    del X
    del y
    del k_fold
    del clf
    results = {
        "acc" : np.mean(acc),
        "f1_w" : np.mean(f1),
        "precision_w" : np.mean(pre),
        "recall_w" : np.mean(rec),
    }
    return results
    
    print(accuracy_score(y_test, predicted))
    print(classification_report(y_test, predicted))

  
    train = data()
# print(model(train))

# for train_indices, test_indices in k_fold.split(X):
#   clf.fit(X[train_indices],y[train_indices])

with open('results.txt', 'a') as f:
    f.write("<begin>{}\n".format(time.time()))
    f.flush()
    reg=6
    dropout=45
    neurons1=1030
    neurons2=510
    #F1=0.9518510950345215

    #for reg in range(5, 2, -1):
    #for neurons2 in range(256, 1536, 256):
    #for neurons1 in range(256, 1536, 256):
    #for dropout in range(0, 50, 10):
    z = model(data(), dropout/100, neurons1, neurons2, 10.**(-reg))

    log = "{},Reg,{},Dropout,{},L1,{},L2,{},F1,{},Acc,{},Precision,{},Recall,{}\n".format(time.strftime("%Y-%m-%d %H:%M"),reg, dropout, neurons1, neurons2, z['f1_w'], z['acc'], z['precision_w'], z['recall_w'])
    print(log)
    f.write(log)
    f.flush()

    f.write("<end>\n")
    f.flush()


/Users/sihambelgadi/Desktop/anaconda/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/sihambelgadi/Desktop/anaconda/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


NameError: name 'df' is not defined